# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
#ed = Website("https://edwarddonner.com")
ed = Website("https://saviano.com")
ed.links

['#content',
 'https://saviano.com/',
 'https://saviano.com/',
 'https://saviano.com/services/',
 'https://saviano.com/paving/',
 'https://saviano.com/grading/',
 'https://saviano.com/excavation/',
 'https://saviano.com/resurfacing/',
 'https://saviano.com/lighting/',
 'https://saviano.com/drainage/',
 'https://saviano.com/fencing/',
 'https://saviano.com/home/consulting/',
 'https://saviano.com/athletic-surfaces/',
 'https://saviano.com/pickleball-courts/',
 'https://saviano.com/tennis-court-construction/',
 'https://saviano.com/basketball-court-construction/',
 'https://saviano.com/running-tracks/',
 'https://saviano.com/cushion-services/',
 'https://saviano.com/about/',
 'https://saviano.com/blog/',
 'https://saviano.com/reviews/',
 'https://saviano.com/submit-a-review/',
 'https://saviano.com/olympics/',
 'https://saviano.com/tennis-booklet/',
 'https://saviano.com/global/',
 'https://saviano.com/request-a-quote/',
 'https://saviano.com/contact/',
 'https://saviano.com/services/',


## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://saviano.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#content
https://saviano.com/
https://saviano.com/
https://saviano.com/services/
https://saviano.com/paving/
https://saviano.com/grading/
https://saviano.com/excavation/
https://saviano.com/resurfacing/
https://saviano.com/lighting/
https://saviano.com/drainage/
https://saviano.com/fencing/
https://saviano.com/home/consulting/
https://saviano.com/athletic-surfaces/
https://saviano.com/pickleball-courts/
https://saviano.com/tennis-court-construction/
https://saviano.com/basketball-court-construction/
https://saviano.com/running-tracks/
https://saviano.com/cushion-services/
https://saviano.com/about/
https://saviano.com/blog/
https://saviano.com/reviews/
https://saviano.com/submit-a-review/
https://s

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/inference-providers',
 '/deepseek-ai/DeepSeek-R1',
 '/deepseek-ai/Janus-Pro-7B',
 '/deepseek-ai/DeepSeek-V3',
 '/mistralai/Mistral-Small-24B-Instruct-2501',
 '/unsloth/DeepSeek-R1-GGUF',
 '/models',
 '/spaces/deepseek-ai/Janus-Pro-7B',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/Qwen/Qwen2.5-Max-Demo',
 '/spaces/Mistral-AI-Game-Jam/NeuralJam',
 '/spaces',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/cognitivecomputations/dolphin-r1',
 '/datasets/ServiceNow-AI/R1-Distill-SFT',
 '/datasets/bespokelabs/Bespoke-Stratos-17k',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/micros

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'Github page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Welcome to Inference Providers on the Hub 🔥
smolagents - a smol library to build great agents
Use models from the HF Hub in LM Studio
The AI community building the future.
The platform where the machine learning community collaborates o

In [41]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short humorous, entertaining brochure about the company for prospective and previous customers.
Include details of company culture and customers if you have the information. Focus on work to create or maintain pickle ball courts.
Write in a theme for Valentine's Day.
Emphasize customers that may need upkeep and maintenance of their existing property and installations.
Return results in Markdown. Limit response to 250 words.
"""


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nWelcome to Inference Providers on the Hub 🔥\nsmolagents - a smol library to build great agents\nUse models from the HF Hub in LM Studio\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1\nUpdated\n2 days ago\n•\n845k\n•\n6.06k\ndeepseek-ai/Janus-Pro-7B\nUpdated\n1 day ago\n•\n134k\n•\n2.4k\ndeepseek-ai/DeepSeek-V3\nUpdated\n10 days ago\n•\n877k\n•\n3.02k\nmistralai/Mistral-Small-24B-Instruct-2501\nUpdated\nabout 12 hours ago\n•\n12.1k\n•\n492\nunsloth/D

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [30]:
create_brochure("Saviano, Inc.", "https://saviano.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://saviano.com/about/'}, {'type': 'services page', 'url': 'https://saviano.com/services/'}, {'type': 'contact page', 'url': 'https://saviano.com/contact/'}, {'type': 'request a quote page', 'url': 'https://saviano.com/request-a-quote/'}]}


```markdown
# 💖 Fall in Love with Your Courts This Valentine’s Day! 💖

Welcome to **Saviano Co. Inc.**, where our love for court construction is as timeless as your favorite romance! Since 1963, we've been crafting dreamy courts that make hearts skip a beat and tennis balls bounce gloriously. We're not just experts; we're your court construction soulmates! 💘

## 🏗️ Our Heartfelt Services
Whether you need to sweep your courts off their feet or give them a little Valentine’s Day makeover, our loveable range of services includes:

- **Paving**: Like a smooth first date, we set the perfect foundation! 
- **Grading & Excavation**: We dig deep to find and love every inch of your property.
- **Resurfacing**: Give your courts the glow-up they deserve. It's basically a spa day for your tennis surface! 
- **Lighting**: Because love should shine bright, even at night.
- **Fencing**: Protect your court, just as you’d protect your heart.
- **Consulting**: Let's talk about your dreams over a virtual cup of cocoa. ☕

And not to forget our **Athletic Surfaces**- perfect for Tennis, Pickleball, Basketball, and even Running Tracks! We cater to players who love to play until the sun sets. 

## 🌍 A Family Legacy 
The Saviano family has served up love and loyalty for generations! With nine Wimbledon contenders in the family, we know how to keep your game strong and maintain high standards. We believe in relationships built on trust, just as John Saviano trusts his court-building passion. 

## 💌 Our Customers, Our Sweethearts
We love our customers, and they love us back! Our portfolio is filled with passionate players and property owners who want to maintain that little slice of court paradise you’ve got tucked away. Whether you run hotel chains or are a suburban homeowner needing a little TLC for your court, we’re here to turn that love for the game into something beautiful.

### So, you're feeling a little *maintenance on your heart*? 💔
No worries! We specialize in **upkeep and maintenance**. With us, your surfaces will be so smooth you'll think they're wearing Cupid’s wings. 

## 💼 Join Our Love Parade! 
Looking for a career that makes you feel like you could serve an ace every day? **Saviano Co. Inc.** is hiring! Become part of our family, and let’s create something beautiful together. Because love isn’t just a feeling – it’s a career choice!

### Contact Us
Ready to explore your court’s potential with **Saviano Co. Inc.**? Give us a call at **(800) 81-TENNIS** or email us at **info@saviano.com** for a chat that might lead to love! 

### Celebrate Love!
This Valentine's Day, choose us - your trusted partner in court construction and maintenance. Because great courts deserve wonderful care! 💞

---
*Saviano Co. Inc. – Where Passion Meets Court Construction!*
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [44]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True,
        temperature=0.2
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [45]:
stream_brochure("Saviano, Inc.", "https://saviano.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://saviano.com/about/'}, {'type': 'services page', 'url': 'https://saviano.com/services/'}, {'type': 'contact page', 'url': 'https://saviano.com/contact/'}, {'type': 'blog page', 'url': 'https://saviano.com/blog/'}, {'type': 'reviews page', 'url': 'https://saviano.com/reviews/'}, {'type': 'consulting page', 'url': 'https://saviano.com/home/consulting/'}]}



# 💖 Love at First Serve: Saviano Co. Inc. 💖

**Roses are red,  
Pickleballs are green,  
Saviano builds courts,  
Where love can be seen!**

At Saviano Co. Inc., we’ve been serving up love since 1963! With a family legacy that’s as strong as your backhand, we specialize in constructing and maintaining pickleball courts that will make your heart skip a beat. Whether you’re looking to build a new court or give your existing one a little TLC, we’ve got the expertise to keep your game—and your love life—on point!

### Why Choose Us?
- **Expertise That’s Love-ly**: With decades of experience, our team knows how to create the perfect playing surface for your romantic rallies.
- **Maintenance with Heart**: Is your court looking a little worse for wear? Let us sprinkle some magic dust (and a bit of resurfacing) to keep the love alive!
- **Family Values**: Just like a good relationship, we believe in trust, quality, and a whole lot of fun. 

### Our Services:
- **Paving & Grading**: Lay the foundation for love!
- **Resurfacing**: Because every court deserves a makeover.
- **Lighting & Fencing**: Keep the sparks flying, day or night!

So, whether you’re a couple looking to rekindle your passion for pickleball or a facility needing a little maintenance love, Saviano Co. Inc. is here to serve! 

**Contact us today and let’s make your court the place where love blossoms!**  
📞 (800) 81-TENNIS  
🌐 [saviano.com](http://saviano.com)

**Happy Valentine’s Day!**


In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

In [1]:
#
# # Full code
#

# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

%pip install -q bs4
%pip install -q -U google-generativeai
%pip install -q ollama

import os
import requests
import json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import google.generativeai as genai
import ollama
import gradio as gr

llm2use = 'googleai'

match llm2use:
    case 'openai':
        MODEL = 'gpt-4o-mini'
    case 'googleai':
        MODEL = 'gemini-1.5-pro'
    case 'ollama':
        MODEL = 'llama3.2'
        # Let's just make sure the model is loaded
        !ollama pull llama3.2


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
#anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
#if anthropic_api_key:
#   print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
#else:
#    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyDi


In [3]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()
#claude = anthropic.Anthropic()
genai.configure(api_key=google_api_key)

In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [7]:
def get_links(url):
    website = Website(url)
   
    match llm2use:
        case 'openai':
            response = openai.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": link_system_prompt},
                    {"role": "user", "content": get_links_user_prompt(website)}
            ],
                response_format={"type": "json_object"}
            )
            result = response.choices[0].message.content
        case 'ollama':
            response = ollama.chat(
                model=MODEL,
                messages=[
                    {"role": "system", "content": link_system_prompt},
                    {"role": "user", "content": get_links_user_prompt(website)}
                ]
            )
            result = response['message']['content']
        case 'googleai':
            model=genai.GenerativeModel(
                model_name=MODEL,
                system_instruction=link_system_prompt)
            response = model.generate_content(
                get_links_user_prompt(website),
                generation_config=genai.GenerationConfig(
                    response_mime_type="application/json"
                ),
            )
            result = response.text

    return json.loads(result)

In [8]:
# get_links("https://saviano.com")

In [9]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    #print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
# get_all_details("https://saviano.com")

In [11]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short humorous, entertaining brochure about the company for prospective and previous customers.
Include details of company culture and customers if you have the information. Focus on work to create or maintain pickle ball courts.
Write in a theme for Valentine's Day.
Emphasize customers that may need upkeep and maintenance of their existing property and installations.
Return results in Markdown.
"""

In [12]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [13]:
def create_brochure(company_name, url):
    temperature = 0.2
    print("Using %s LLM with model %s and temperature %.2f" % (llm2use,MODEL,temperature))

    match llm2use:
        case 'openai':
            response = openai.chat.completions.create(
                model=MODEL,
                temperature=temperature,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
                ],
            )
            result = response.choices[0].message.content
        case 'ollama':
            response = ollama.chat(
                model=MODEL,
                options = {
                    #'temperature': 1.5, # very creative
                    'temperature': temperature
                },         
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
                ]
            )
            result = response['message']['content']
        case 'googleai':
            model=genai.GenerativeModel(
                model_name=MODEL,
                system_instruction=system_prompt)
            response = model.generate_content(
                get_brochure_user_prompt(company_name, url),
                generation_config = genai.GenerationConfig(
                    temperature=temperature
                )
            )
            result = response.text

    display(Markdown(result))

In [14]:
# create_brochure("Saviano, Inc.", "https://saviano.com")

In [15]:
#
# # Everything below here uses values from the gradio web interface. Not the static values defined above
#
def stream_openai(system_prompt,company_name,url,model,temperature):

    response = openai.chat.completions.create(
        model=model,
        stream=True,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )

    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        yield result

    print(result)
    

In [16]:
def stream_googleai(system_prompt,company_name,url,model,temperature):

    model=genai.GenerativeModel(
        model_name=model,
        system_instruction=system_prompt)
    responses = model.generate_content(
        get_brochure_user_prompt(company_name, url),
        generation_config = genai.GenerationConfig(
            temperature=temperature
        ),
        stream=True
    )

    result = ""
    for response in responses:
        result += response.text
        yield result

    print(result)


In [17]:
def stream_ollama(system_prompt,company_name,url,model,temperature):

    response = ollama.chat(
        stream=True,
        model=model,
        options = {
            #'temperature': 1.5, # very creative
            'temperature': temperature
        },         
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ]
    )

    # Printing out each piece of the generated response while preserving order
    result = ""
    for chunk in response:
        result += chunk['message']['content']
        yield result

    print(result)


In [18]:
def stream_model(system_prompt,company_name,url,llm_model,temperature):

    print("Analyzing %s from %s" % (company_name,url))

    match llm_model:
        case 'openai_gpt-4o-mini':
            model = "gpt-4o-mini"
            print("\n*** Using LLM %s and model %s\n" % ("openai",model))
            result = stream_openai(system_prompt,company_name,url,model,temperature)
        case 'googleai_gemini-1.5-pro':
            model = "gemini-1.5-pro"
            print("\n***Using LLM %s and model %s\n" % ("googleai",model))
            result = stream_googleai(system_prompt,company_name,url,model,temperature)
        case 'googleai_gemini-1.5-flash':
            model = "gemini-1.5-flash"
            print("\n***Using LLM %s and model %s\n" % ("googleai",model))
            result = stream_googleai(system_prompt,company_name,url,model,temperature)
        case 'ollama_llama3.2':
            model = "llama3.2"
            print("\n***Using LLM %s and model %s\n" % ("ollama",model))
            result = stream_ollama(system_prompt,company_name,url,model,temperature)
        case _:
            raise ValueError("Unknown LLM and MODEL combination")
        
        
    yield from result

In [19]:
view = gr.Interface(
    fn=stream_model,
    inputs=[
        gr.Textbox(label="AI Agent",value=system_prompt),
        gr.Textbox(label="Company Name", value="Saviano, Co."),
        gr.Textbox(label="Website to analyze", value="https://saviano.com"),
        gr.Dropdown(["openai_gpt-4o-mini", "googleai_gemini-1.5-pro", "googleai_gemini-1.5-flash","ollama_llama3.2"], label="Select AI Engine and LLM Model", value="ollama_llama3.2"),
        gr.Slider(0,2,value=0.7,label="AI Temperature",info="AI Creativity Level (higher = more creative)")
    ],
    outputs=[
        gr.Markdown(label="Response:")
    ],
    flagging_mode="never"
)
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://db5c1425d7fa6f8b83.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Analyzing Saviano, Co. from https://saviano.com

***Using LLM ollama and model llama3.2

**Love is in the Air... and on the Court!**

Welcome to Saviano Co. Inc., your premier partner for pickleball court construction, maintenance, and everything in between!

**A Family Legacy of Excellence**
---------------------------------

At Saviano Co. Inc., we're not just a company - we're a family with a passion for tennis and a commitment to excellence. Since 1963, we've been building top-tier athletic courts for pickleball, tennis, basketball, and more.

**Why Choose Us?**
------------------

* **Expert Craftsmanship**: Our team of professionals delivers exceptional construction services with speed, precision, and care.
* **Fair Pricing**: We believe in delivering value without compromising on quality.
* **Guaranteed Satisfaction**: Our customers love what we do, and we're confident you will too!

**Our Customers Say It Best**
------------------------------

* "Saviano Co. Inc. is the go-to p

In [31]:
%pip install -q huggingface_hub

hf_token = 'REPLACE_WITH_ACTUAL_TOKEN'
target_space_name = 'create-brochure'
path_to_file = 'week1'

from huggingface_hub import (
    create_repo,
    get_full_repo_name,
    upload_file,
)
#create_repo(name=target_space_name, token=hf_token, repo_type="space", space_sdk="gradio")
repo_name = get_full_repo_name(model_id=target_space_name, token=hf_token)
file_url = upload_file(
    path_or_fileobj=path_to_file,
    path_in_repo="app.py",
    repo_id=repo_name,
    repo_type="space",
    token=hf_token,
)

Note: you may need to restart the kernel to use updated packages.


ValueError: Provided path: 'week1' is not a file on the local file system

Analyzing Saviano, Co. from https://saviano.com

***Using LLM ollama and model llama3.2

**Love is in the Air... and on the Court!**

Welcome to Saviano Co. Inc., the ultimate partner for all your court construction needs. As we celebrate Valentine's Day, we're reminded of the importance of love, care, and attention to detail - qualities that our team embodies in every project we undertake.

**A Family Affair**

At Saviano Co. Inc., we're a family company with a legacy that spans over 60 years. Our passion for tennis and commitment to excellence have been passed down through generations, ensuring that every project is crafted with precision and care.

**Pickleball Perfection**

We specialize in top-tier athletic courts for pickleball, tennis, basketball, and more. Our expert craftsmanship, fair pricing, and guaranteed satisfaction ensure that your court is a haven for players of all levels. Whether you're looking to improve your game or create unforgettable tennis experiences, our team

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>